In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# batch size
batch_size = 100
# 计算有多少批次 
n_batch = mnist.train.num_examples // batch_size


W0828 22:11:03.391719 26608 deprecation.py:323] From <ipython-input-2-3d105be9da94>:1: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0828 22:11:03.394725 26608 deprecation.py:323] From H:\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0828 22:11:03.449701 26608 deprecation.py:323] From H:\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\base.py:252: _internal_retry.<locals>.wrap.<locals>.wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updatin

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data\train-images-idx3-ubyte.gz


W0828 22:11:06.852611 26608 deprecation.py:323] From H:\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0828 22:11:06.855620 26608 deprecation.py:323] From H:\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.


Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz


W0828 22:11:08.075219 26608 deprecation.py:323] From H:\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [5]:
# 初始化权值
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1) # 生成一个截断的正态分布
    return tf.Variable(initial)
#初始化偏置
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)
# 卷积层
def conv2d(x,W):
    # x input tensor of shape `[batch, in_height, in_width, in_channels]`
    # W filter/kernel tensor of shape [filter_height, filter_width, in_channels, out_channels]
    # strides[0] = strides[3] = 1    strides[1]代表x方向步长 strides[2]代表y方向步长
    # padding A `string`from"'SAME' 'VALID'
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')
# 池化层
def max_pool_2x2(x):
    # kseze [1,x,y,1] 窗口大小 1 2位置可以设置
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

# 定义两个placeholder
x = tf.placeholder(tf.float32, [None,784])
y = tf.placeholder(tf.float32, [None,10])

# 改变x的格式转为4d的向量  相当于复原 ，-1会变成100，batch size， 1 因为是黑白图片
x_image = tf.reshape(x, [-1,28,28,1])

# 初始化第一个卷积层的权重和偏置
W_conv1 = weight_variable([5,5,1,32]) # 5*5的采样窗口 32个卷积核从一个平面抽取特征
b_conv1 = bias_variable([32]) # 32个卷积核，每一个卷积核给一个偏置值

# 把x_image 和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1) # 进行max_pooling

# 初始化第二个卷积层的权值和偏置
W_conv2 = weight_variable([5,5,32,64]) # 5*5采样窗口， 64个卷积核从32个平面抽取特征
b_conv2 = bias_variable([64]) # 每个卷积核 给一个偏置值

# 把h_pool1 和权值向量进行卷积 再加上偏置值，然后应用于relu激活函数
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2) # max_pooling 2*2

# 28*28的图片第一次卷积后还是28*28，第一次池化后为14*14
# 第二次卷积后为14*14，第二次池化后变为7*7
# 经过上面操作后 得到64张7*7的平面

# 初始化第一个全连接层的权值
W_fc1 = weight_variable([7*7*64,1024]) # 上一层有7*7*64 个神经元，全连接层有1024个神经元
b_fc1 = bias_variable([1024]) # 1024个节点

# 池化层2的输出扁平化为1维
h_pool2_flat = tf.reshape(h_pool2, [-1,7*7*64])
# 求第一个全连接层输出
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+ b_fc1)

# keep_prob 用来表示神经元的输出概率
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# 初始化第二个全连接层
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])

# 计算输出
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2) + b_fc2)

# 交叉熵代价函数
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits = prediction))

# 使用AdamOptimizer进行优化
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
# 结果放在一个布尔列表中
correct_prediction = tf.equal(tf.argmax(prediction,1),tf.arg_max(y,1)) # argmax返回一维张量中最大的值所在的位置
# 求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x:batch_xs, y:batch_ys, keep_prob:0.7})
    
    acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels, keep_prob:1.0})
    print('Iter'+ str(epoch) + ' Testing Accuracy = ' + str(acc))





W0828 22:59:31.166761 26608 deprecation.py:323] From <ipython-input-5-cb1094e76b14>:74: arg_max (from tensorflow.python.ops.gen_math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.math.argmax` instead


Iter20 Testing Accuracy = 0.9909
